In [1]:
import os

In [4]:
%pwd

'c:\\Users\\youne\\OneDrive\\Desktop\\End-to-End-Cardiovascular-Disease'

In [3]:
os.chdir('../')

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [6]:
from Cardiovascular_Disease.constants import *
from Cardiovascular_Disease.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [19]:
import os
from Cardiovascular_Disease import logger
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd

In [26]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        

    def label_encode(self):
        le = LabelEncoder()
        data = pd.read_csv(self.config.data_path)
        for col in ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']:
                data[col] = le.fit_transform(data[col])
        logger.info("Label encoding completed.")
        return data  
 
    
    def scale(self,data):
        mms = MinMaxScaler() # Normalization
        ss = StandardScaler() # Standardization
        data['Oldpeak'] = mms.fit_transform(data[['Oldpeak']])
        data['Age'] = ss.fit_transform(data[['Age']])
        data['RestingBP'] = ss.fit_transform(data[['RestingBP']])
        data['Cholesterol'] = ss.fit_transform(data[['Cholesterol']])
        data['MaxHR'] = ss.fit_transform(data[['MaxHR']])
        logger.info("Scaling completed.")
        return data 


    def feature_engineering(self,data):
         features = data[data.columns.drop(['HeartDisease','RestingBP','RestingECG'])].values
         target = data['HeartDisease'].values
         



    def train_test_spliting(self,data):
        

        # Split the data into training and test sets. (0.75, 0.25) split.
        train, test = train_test_split(data)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)

In [27]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    encoded_data = data_transformation.label_encode()
    scaled_data = data_transformation.scale(encoded_data)
    data_transformation.train_test_spliting(scaled_data)
except Exception as e:
    raise e

[2024-10-22 22:56:22,682: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-22 22:56:22,684: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-22 22:56:22,689: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-10-22 22:56:22,691: INFO: common: created directory at: artifacts]
[2024-10-22 22:56:22,693: INFO: common: created directory at: artifacts/data_transformation]
[2024-10-22 22:56:22,703: INFO: 2404743328: Label encoding completed.]
[2024-10-22 22:56:22,726: INFO: 2404743328: Scaling completed.]
[2024-10-22 22:56:22,742: INFO: 2404743328: Splited data into training and test sets]
[2024-10-22 22:56:22,744: INFO: 2404743328: (688, 12)]
[2024-10-22 22:56:22,745: INFO: 2404743328: (230, 12)]
(688, 12)
(230, 12)
